<a href="https://colab.research.google.com/github/elianedp/ExpertSystem/blob/main/%5BONLINE%5D_chatbot_faq_bank_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot de FAQ bancário
## Tecnólogo em Inteligência Artificial Aplicada - Agentes Conversacionais
Neste notebook iremos trabalhar com um chatbot utilizado para responder perguntas do FAQ (Frequently Asked Questions) de um banco.

### Implementação utilizada - [Chatbot](https://github.com/kunkaweb/Chatbot)
Este chatbot é **híbrido**, que utiliza-se de regras e dados em um corpus. Ele faz uso de tecnicas básicas de Processamento de Linguagem Natural, implementadas nas bibliotecas **nltk** e **scikit-learn**.

# ============================================================
#** ATIVIDADE FORMATIVA – Complementando um agente de perguntas frequentes**
#
# Aluna: Eliane Maria de Pieri
# Curso: Tecnólogo em Inteligência Artificial Aplicada
# Disciplina: Agentes Conversacionais
#
# MELHORIA IMPLEMENTADA (Opção 2 da atividade):
# Armazenar todas as listagens de possíveis respostas em arquivos externos,
# evitando manter respostas fixas no código.
#
# ✅ Alterações realizadas:
# 1. Criado o arquivo externo "respostas.csv" contendo respostas para:
#    - Saudações
#    - Agradecimentos
#    - Despedidas
#
# 2. Substituídas as funções originais:
#    - greeting() → agora busca respostas da categoria "saudacao" no CSV
#    - isThanks() → agora busca respostas da categoria "agradecimento" no CSV
#    - isEnding() → agora busca respostas da categoria "despedida" no CSV
#
# 3. Adaptado o laço principal do chatbot para trabalhar com as novas funções,
#    recebendo as respostas diretamente do arquivo externo.
#
# 4. Mantidos os demais blocos (normalização, corpus FAQ, TF-IDF, etc.),
#    pois já utilizam dados externos (faq-corpus.txt).
#
# Resultado:
# O chatbot agora pode ser facilmente expandido ou atualizado apenas editando
# o arquivo "respostas.csv", sem necessidade de alterar o código-fonte.
# ============================================================

In [19]:
import nltk
import numpy as np
import random
import string
import warnings
import pandas as pd

# Conecta ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Lê as respostas direto do Drive
respostas_df = pd.read_csv("/content/drive/MyDrive/chatbot/respostas.csv")

# Função para escolher resposta de uma categoria
def responder_categoria(categoria):
    possiveis = respostas_df[respostas_df["categoria"] == categoria]["resposta"].tolist()
    if possiveis:
        return random.choice(possiveis)
    else:
        return "Desculpe, não tenho resposta para isso."


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import os
print(os.listdir())

['.config', 'drive', 'respostas.csv', 'sample_data']


#### Carregando corpus do FAQ
Aqui nós carregamos um corpus com as perguntas frequentes do banco, disponíveis em arquivo texto. Este documento pode ser editado a qualquer momento, seja para inserir novas questões, ou apagar e editar questões já existentes.

In [ ]:
# Abre arquivo e coloca em minúsculas
f = open('faq-corpus.txt','r')
raw = f.read().lower()

# Segmenta sentenças e tokeniza o corpus
sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [ ]:
sent_tokens

['você pode abrir sua conta em uma de nossas agências ou unidades do correspondente mais bb e banco postal.',
 'menores de 18 anos (não emancipados) deverão comparecer à agência acompanhados do responsável legal, munidos dos respectivos documentos.',
 'os documentos necessários são cpf, documento de identificação original com foto, comprovante de endereço e renda atualizados (até 90 dias), caso não possua comprovante de residência no próprio nome, é aceita declaração do titular do comprovante com firma reconhecida em cartório.',
 'é necessário que você e as pessoas que serão incluídas compareçam à agência, todos os titulares devem apresentar: cpf, documento de identificação original com foto, comprovantes de endereço e renda atualizados (ambos com validade de até 90 dias).',
 'sim, será necessário que todos assinem documento específico disponibilizado na agência que mantém a conta ou que estejam presentes e façam a concordância formal mediante carta própria assinada por todos.',
 'toma

In [ ]:
word_tokens[0:15]

['você',
 'pode',
 'abrir',
 'sua',
 'conta',
 'em',
 'uma',
 'de',
 'nossas',
 'agências',
 'ou',
 'unidades',
 'do',
 'correspondente',
 'mais']

#### Normalização e redução de dimensionalidade
Aqui definimos funções que utilizam o processo de Stemming nos textos, para normalizar termos similares a uma única forma. Exemplo: "gato", "gatos", "gata" e "gatas" são normalizados para "gat".

In [ ]:
stemmer = nltk.stem.RSLPStemmer()

def LemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

#### Controle de saudações
Define-se uma função para lidar com saudações feitas ao chatbot

In [ ]:
# Função de controle de saudações adaptada para usar respostas do CSV
# Em vez de manter as respostas fixas no código, buscamos no arquivo respostas.csv
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in ["olá", "oi", "ei", "blz", "bom dia", "boa tarde", "boa noite"]:
            # Agora pegamos uma resposta aleatória da categoria "saudacao"
            return responder_categoria("saudacao")

#### Controle de agradecimento
Define uma função para verificar se é um agradecimento do usuário

In [ ]:
# Função de controle de agradecimentos adaptada para usar respostas do CSV
# Antes: apenas verificava se era agradecimento e retornava True/False
# Agora: retorna diretamente uma resposta da categoria "agradecimento"
def isThanks(sentence):
    for word in sentence.split():
        if word.lower() in ["obrigado", "muito obrigado", "obrigada", "muito obrigada", "agradecido", "agradecida"]:
            return responder_categoria("agradecimento")
    return None  # Se não encontrar, não retorna nada

#### Controle de despedidas
Função para verificar se é uma despedida do usuário

In [ ]:
# Função de controle de despedidas adaptada para usar respostas do CSV
# Antes: apenas verificava e retornava True/False
# Agora: retorna diretamente uma resposta da categoria "despedida"
def isEnding(sentence):
    for word in sentence.split():
        if word.lower() in ["tchau", "até logo", "fui", "adeus", "até mais"]:
            return responder_categoria("despedida")
    return None  # Se não encontrar, não retorna nada

#### Controle de respostas
Definimos uma função para controlar as respostas dadas ao usuário. Para tal, utilizamos a métrica do **TF-IDF** e a **similaridade de cosseno**, para encontrar em nosso corpus a resposta que mais se encaixa na pergunta feita pelo usuário, ou seja, **a resposta cujo texto seja mais similar ao texto da pergunta**.

In [ ]:
# Esse bloco do "Controle de respostas" permanece como está,
# porque já utiliza um corpus externo (faq-corpus.txt).
# Não precisamos adaptá-lo, pois a formativa pediu apenas
# para externalizar as respostas fixas (saudações, agradecimentos e despedidas).

def response(user_response):
    warnings.simplefilter('ignore')
    robo_response=''

    # Cria uma lista contendo todas sentenças do corpus + a sentença enviada pelo usuário. Assim geramos o TF-IDF de tudo.
    sentencas = sent_tokens + [user_response]

    # Inicia tranformação do texto em uma matriz TF-IDF - normaliza texto e retira stop-words
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=nltk.corpus.stopwords.words('portuguese'))
    tfidf = TfidfVec.fit_transform(sentencas)

    # Efetua cálculo de similaridade de cosseno entre o texto do usuário, e as sentenças do corpus
    vals = cosine_similarity(tfidf[-1], tfidf)
    # Obtém índice/posição da termo com maior grau de similaridade com o texto do usuário
    idx = vals.argsort()[0][-2]
    # Ordena vetor de similaridade
    flat = vals.flatten()
    flat.sort()
    # Obtém o valor de similaridade do penúltimo item no vetor
    # Pois o último item é o próprio texto do usuário, que obtém sempre grau máximo de similaridade
    # Portanto, obtemos a segunda maior similaridade, pois não queremos o próprio texto enviado pelo usuário
    req_tfidf = flat[-2]

    if IS_DEBUG:
      print("Vetor de similaridade (último valor é o próprio texto do usuário): ")
      print(vals)
      print("Índice da maior similaridade: ")
      print(idx)
      print("Vetor de similaridade ordenado: ")
      print(flat)
      print("Similaridade da sentença mais similar encontrada: ")
      print(req_tfidf)

    # Caso não seja encontrada similaridade para nenhum item do corpus
    if(req_tfidf==0):
        robo_response=robo_response+"Você poderia reformular sua pergunta?"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

#### Executando o chatbot
Trecho de código que controla a aplicação de maneira geral. A seguir algumas perguntas que você pode fazer ao chatbot:



1.   Como faço para abrir uma conta corrente?
2.   Quais são os documentos necessários?
3.   Do que devo tomar conhecimento?
4.   Quero incluir outra pessoa como titular em minha conta, o que devo fazer?


In [ ]:
flag=True

print("FAQBOT: Olá, eu sou o faqbot. Posso responder suas perguntas referentes a abertura de conta no banco.")

while(flag==True):
    # Obtém entrada do usuário
    user_response = input()
    user_response = user_response.lower()

    # Verifica se é uma despedida
    end_resp = isEnding(user_response)
    if end_resp:  # Se encontrou uma despedida no CSV
        flag = False
        print("FAQBOT: " + end_resp)

    else:
        # Verifica se é agradecimento
        thanks_resp = isThanks(user_response)
        if thanks_resp:  # Se encontrou agradecimento no CSV
            flag = False
            print("FAQBOT: " + thanks_resp)

        else:
            # Verifica se é saudação
            greet_resp = greeting(user_response)
            if greet_resp:  # Se encontrou saudação no CSV
                print("FAQBOT: " + greet_resp)

            else:
                # Caso contrário, responde com o FAQ (TF-IDF + cosseno)
                print("FAQBOT: ", end="")
                print('\x1b[1;31m' + response(user_response) + '\x1b[0m')

### O que mais pode ser feito?

- Poderíamos organizar melhor nosso chatbot ao utilizar uma **função única para gerenciamento de diálogos**, assim poderíamos remover as funções de controle de saudações, agradecimento e despedidas.
- Armazenar todas listagens de possíveis respostas em **arquivos externos** (como foi feito com o corpus do FAQ), assim, evitamos ter de editar o código sempre que precisarmos adicionar novas repostas.
- Atualmente o corpus de FAQ contém apenas as respostas possíveis. Poderíamos adicionar a ele também as perguntas relativas a cada resposta, assim, verificariamos a similaridade entre a pergunta feita pelo usuário e a pergunta do FAQ, e não com a possível resposta. Com isso espera-se que o chatbot seja mais preciso ao encontrar respostas, pois a similaridade encontrada tende a ser maior.
- Utilizar *web scraping* para montar corpus de FAQ automaticamente, ao ler todas perguntas e respostas diretamente do site do banco (e.g., [FAQ do BB](https://www.bb.com.br/pbb/pagina-inicial/perguntas-frequentes#/))


## Referências e Material complementar

* [Chatbot simples com NLTK e scikit-learn](https://github.com/kunkaweb/Chatbot)
* [scikit-learn - TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
* [scikit-learn - cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).